In [ ]:
import numpy as np
import pandas as pd  
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
import pyarrow.feather as feather
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
!pip install shap
import shap
!pip install plotly
import warnings
warnings.filterwarnings('ignore')
! brew install graphviz
RSEED= 12

# Exporting the prepared dataframe and changing the format of a columns that was causing problems

In [ ]:
export_df = feather.read_feather("../data/cleaned_data.feather")
export_df['lenght']=[row.astype(int) for row in export_df['lenght']]

# Functions for running XGBoost

In [ ]:
# Function to split the dataset 
def splitdataset(df):
    ''' 
    Function for spliting the dataset, aiming four our target and stratifying the data.
    Return train and test variables.
    '''
    y=export_df["interesting_message"]
    X=export_df.drop("interesting_message",axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, test_size = 0.2,random_state = 42)
    return X_train, X_test, y_train, y_test

In [ ]:
def train_tree(X_train, y_train):
    '''
    Function for training the decision tree, it includes hyperparameter tuning and cross validation.
    Returns best model achieved.
    '''
    
    # Creating the classifier object 
    reg_tree = DecisionTreeClassifier(class_weight="balanced")
    param_tree = {'max_leaf_nodes':[20,50,100,500],
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 5, 10, 50],
    'min_samples_leaf':[50,200,500,1000]
    }

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    grid_tree = GridSearchCV(reg_tree, param_grid=param_tree, cv=cv,scoring='balanced_accuracy', n_jobs=-1, verbose=10)
    grid_tree.fit(X_train, y_train)
    print('Best score:\n{:.2f}'.format(grid_tree.best_score_))
    print("Best parameters:\n{}".format(grid_tree.best_params_))
    print("Best model_tree:\n{}".format(grid_tree.best_estimator_))
    best_model_tree = grid_tree.best_estimator_

    return best_model_tree

In [ ]:
# Function to make predictions 
def prediction(X_test, reg_tree):
    '''
    Function for predicting classification with input model.
    Return prediction values.
    '''

    y_pred = reg_tree.predict(X_test)
    return y_pred 

In [ ]:
# Function to calculate accuracy 
def class_metrics(y_test, y_pred):
    
    '''
    Function to evaluate results achieved on prediction.
    Returns confusion matrix and balanced accuracy.
    '''

    accuracy = balanced_accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred).round()
    print("Predicted values:\n", y_pred) 
    print("Confusion Matrix: \n", cm) 
    print("Balanced Accuracy: %.4f%%" % (accuracy * 100.0))
    print("Report : \n", classification_report(y_test, y_pred))

    return cm, accuracy

In [ ]:
def plot_importance (model,X_train):
    '''
    Function for plotting the shap values (importance of features) of the given model.
    It plots the most important values, and returns theses values and the explainer model.
    '''
    explainer=shap.TreeExplainer(model,data=X_train)
    shap_values = explainer.shap_values(X_train)
    shap.summary_plot(shap_values, X_train, plot_type="bar")
    return shap_values,explainer
    #explainer=shap.TreeExplainer(model,data=X_train)
    #shap_values = explainer.shap_values(X_train)
    #vals= np.abs(shap_values).mean(0)
    #feature_importance = pd.DataFrame(list(zip(X_train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
    #ordered_weights= feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
    #shap.summary_plot(shap_values, X_train, plot_type="bar")
    
    #return ordered_weights


In [ ]:
def get_importances (shap_values):
    '''
    Function for getting the shap values (importance of features).
    Prints the head of these values and returns the values in descending order of importance.
    '''
    vals= np.abs(shap_values).mean(0)
    feature_importance = pd.DataFrame(list(zip(X_train.columns,vals)),columns=['col_name','feature_importance_vals'])
    feature_importance.sort_values(by=['feature_importance_vals'],ascending=False,inplace=True)
    feature_importance.head()
    return feature_importance

In [ ]:
def main():
    '''
    Main for running the decision tree modeling.
    Returns base model, prediction, confusion matrix, balanced accuracy and the features split for the importance analysis.
    '''
    # Building Phase 
    X_train, X_test, y_train, y_test = splitdataset(export_df) 
    basemodel = train_tree(X_train, y_train)
    print(f'Decision tree has {basemodel.tree_.node_count} nodes with maximum depth {basemodel.tree_.max_depth}.')
    # Operational Phase 
    print("-----"*15)
    print("Results:\n")
    # Prediction
    y_pred = prediction(X_test, basemodel) 
    cm,accuracy= class_metrics(y_test, y_pred)
    return basemodel,y_pred,cm,accuracy,X_train

# Running decision tree

In [ ]:
basemodel,y_pred,cm,accuracy,X_train=main()

# Getting the importance of the features

In [ ]:
ordered_weights=plot_importance(basemodel,X_train)


# Creating the decision tree as an image

In [ ]:
from sklearn.tree import export_graphviz  
features=['lenght', 'emoji_size', 'slang_char', 'slang_verb', 'slang_pron',
       'slang_adp', 'slang_noun', 'slang_num', 'slang_punt', 'slang_det',
       'info_char', 'info_words', 'info_verb', 'info_pron', 'info_adp',
       'info_noun', 'info_num', 'info_punt', 'info_det',
       'information_sentiment_negative', 'information_sentiment_neutral',
       'information_sentiment_positive', 'slang_sentiment_negative',
       'slang_sentiment_neutral', 'slang_sentiment_positive',
       'cause_BrokenVehicle', 'cause_COVID19',
       'cause_Counterflow', 'cause_CycleRide', 'cause_Demonstration',
       'cause_EmergencyServices', 'cause_Event', 'cause_Explosion',
       'cause_FallenTree', 'cause_Fire', 'cause_Flood', 'cause_GasLeak',
       'cause_HeavyTraffic', 'cause_Incident', 'cause_Landslide', 'cause_Leak',
       'cause_Maintenance', 'cause_March', 'cause_Overturn',
       'cause_Pilgrimage', 'cause_ProtestCamp', 'cause_Rain',
       'cause_Reopening', 'cause_Sinkhole', 'cause_StreetWorks',
       'cause_VehicularAccident', 'cause_Waterlogging',
       'effect_CirculationRestored', 'effect_CirculationShutdown',
       'effect_Delays', 'effect_Evacuation', 'effect_FullCapacity',
       'effect_HighWaitingTime', 'effect_InterimService',
       'effect_LaneReduction', 'effect_RouteDetour', 'effect_SecuritySpeed',
       'effect_SuspensionOfService', 'effect_TrafficImpact']
# export the decision tree to a tree.dot file 
#for visualizing the plot easily anywhere 
export_graphviz(basemodel, out_file ='tree_class.dot', 
               feature_names =features) 

In [ ]:
! dot -Tpng tree_class.dot > tree_class.png

# Saving some of the values of our model on a txt file

In [ ]:
f = open('ml-log.txt', 'a')
f.write('Base model: Decision Tree\n Predicted values:\n {}\n Confusion Matrix:\n {}\n Balanced Accuracy:\n {} \n Model: {}'.format(y_pred,cm,accuracy,basemodel))
f.close()